# Periodic Boundary Conditions

In [7]:
import numpy as np
from scipy.sparse import csc_array

import matplotlib.pyplot as plt
import matplotlib as mpl

from qs_mps.applications.Z2.exact_hamiltonian import *
from qs_mps.sparse_hamiltonians_and_operators import diagonalization, sparse_magnetization, sparse_pauli_x, sparse_pauli_z
from qs_mps.mps_class import MPS

## Direct Hamiltonian

In [8]:
def direct_ham(l, L, g):
    # degrees of freedom
    dof = l*(L+1) + l*L
    
    # initialize
    O = csc_array((2**dof, 2**dof), dtype=complex)
    H_sigmas = O
    H_taus = O
    H_plaquettes = O

    # horizontal terms
    for i in range(l*L):
        H_taus += sparse_pauli_x(n=i, L=dof)

    # vertical terms
    for i in range(l*L,l*(L+1)+l*L):
        H_sigmas += sparse_pauli_x(n=i, L=dof)

    # plaquettes terms
    for i in range(l*L-2):
        H_plaquettes += sparse_pauli_z(n=i, L=dof) @ sparse_pauli_z(n=i+2, L=dof) @ sparse_pauli_z(n=i+6, L=dof) @ sparse_pauli_z(n=i+7, L=dof)

    # plaquettes terms last for pbc
    for j in range(L):
        H_plaquettes += sparse_pauli_z(n=i, L=dof) @ sparse_pauli_z(n=j, L=dof) @ sparse_pauli_z(n=i+6, L=dof) @ sparse_pauli_z(n=i+7, L=dof)

    return - g * H_sigmas - g * H_taus - 1/g * H_plaquettes

In [11]:
l = 2
L = 2
g = 1
H_dir = direct_ham(l,L,g)
e, v = diagonalization(H_dir, sparse=True)
print(e)

[-10.67000647]


## Dual Hamiltonian

In [61]:
def dual_ham(l,L,g):
    # degrees of freedom
    dof = l*L + 1

    # charges
    charges = np.ones((l,L+1))
    prod_charges = [1] + np.prod(charges[1:], axis=1).tolist()

    # initialize
    O = csc_array((2**dof, 2**dof), dtype=complex)
    H_sigmas = O
    H_taus = O
    H_plaquettes = O

    # first column of sigma are local
    for i in range(l):
        H_sigmas += sparse_pauli_z(n=i*L, L=dof)
    
    # horizontal zz interactions, we exclude the last column of sigmas
    for j in range(l):
        for i in range(L-1):
            H_sigmas += sparse_pauli_z(n=i+j*L, L=dof) @ sparse_pauli_z(n=(i+1)+j*L, L=dof)

    # horizontal zz interactions, last column of sigmas
    for j in range(l):
        H_sigmas += np.prod(prod_charges[:(j+1)]) * sparse_pauli_z(n=j*L+L-1, L=dof) @ sparse_pauli_z(n=l*L, L=dof)

    # vertical zz interactions 
    for j in range(L):
        for i in range(l):
            np.prod(charges[(i+1)%l,:(j+1)]) * sparse_pauli_z(n=i*L+j, L=dof) @ sparse_pauli_z(n=((i+1)%l)*L+j, L=dof)

    # plaquette terms
    for i in range(l*L):
        H_plaquettes += sparse_pauli_x(n=i, L=dof)

    return - g * H_sigmas - g * H_taus - 1/g * H_plaquettes

In [65]:
L = 2
l = 2
g = 1
H_dual = dual_ham(l,L,g)
e,v = diagonalization(H_dual, sparse=True)
print(e)


[-6.98791841]
